# Natural Language Processing

Natural Language Processing (NLP) is a branch of artificial intelligence (AI) that deals with written and spoken language. You can use NLP to build solutions that extracting semantic meaning from text or speech, or that formulate meaningful responses in natural language.

## Using the Text Analytics Cognitive Service

Microsoft Azure *cognitive services* includes the *Text Analytics* service, which provides some out-of-the-box NLP capabilities, including the identification of key phrases in text, and the classification of text based on sentiment.

For example, suppose Adventure Works Cycles encourages customers to submit reviews for its cycle rental service. You could use the text Analytics service to summarize the reviews by extracting key phrases, and determine which reviews are positive, and which are negative.

If you have't already done so, create a **Cognitive Services** resource in your Azure subscription:

1. Open the [Azure portal](https://portal.azure.com), signing in with your Microsoft account.
2. Click the **&#65291;Create a resource** button, search for *Cognitive Services*, and create a **Cognitive Services** resource with the following settings:
    - **Name**: *Enter a unique name*.
    - **Subscription**: *Your Azure subscription*.
    - **Location**: *Any available location*.
    - **Pricing tier**: S0
    - **Resource group**: *Create a resource group with a unique name*.
3. Wait for deployment to complete. Then go to your cognitive services resource, and on the **Quick start** page, note the keys and endpoint. You will need these to connect to your cognitive services resource from client applications.

To use your cognitive services resource, client applications need its  endpoint and authentication key:

1. Copy the **Key1** for your resource and paste it in the code below, replacing **YOUR_COG_KEY**.
2. Copy the **endpoint** for your resource and and paste it in the code below, replacing **YOUR_COG_ENDPOINT**.
3. Run the code in the cell below by clicking its green <span style="color:green">&#9655</span> button.

> **Note**: If you restart this notebook, you may need to run this cell again to reinitialize these values.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

### Call the Text Analytics Service to Determine Key Phrases in Reviews

One of the methods provided by the Text Analytics service is the ability to extract a list of key phrases from text documents, which give an insight into the core topics discussed in the document.

Run the following cell view the contents of two reviews.

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# The reviews are in the /data/reviews folder
reviews_folder = os.path.join('data', 'reviews')

# Create a JSON collection of reviews to process as a batch
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "language": "en", "text": review_text}
    reviews.append(review)

# Use the Text Analytics service to analyze key phrases and sentiment
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print('-------------\n', reviews[review_num]['id'])
    print('\nText:', reviews[review_num]['text'])

    # Get the sentiment score for this review
    sentiment = 'positive'
    sentiment_score = sentiment_analysis.documents[review_num].score
    if sentiment_score < 0.5:
        sentiment = 'negative'
    print('\nSentiment: {:.2f} ({})'.format(sentiment_score, sentiment))

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')



## Using the Language Understanding Intelligence Service
Increasingly, we expect computers to be able to use AI in order to understand spoken or typed commands in natural language. For example, Adventure Works Cycles might want to implement office automation that enables employees to control the office environment using voice commands such as "switch on the light" or "put the fan on", and have an AI-powered device understand the command and take appropriate action.

### Create a LUIS App
The Microsoft cognitive services include the Language Understanding Intelligence Service (LUIS), which enables you to define *intents* that are applied to *entities* based on *utterances*.

To implement natural language understanding with LUIS, you must first create an app; and then add entities, intents, and utterances to define the commands you want the app to understand:

1. Open a new browser tab and navigate to https://preview.luis.ai/.
2. Sign in using the Microsoft account associated with your Azure subscription. If this is the first time you have signed into LUIS, you may need to grant the app some permissions to access your account details, and then fill in some information and accept the terms of use.
3. If a message prompting you to complete a tutorial displayed, close it (you can complete this tutorial later - for now, we'll focus on a simpler example).
4. In the **My Apps** page, click **Create** and create a new app with the following settings:
  - **Name**: Office Automation
  - **Culture**: English
  - **Description**: Adventure Works office automation
  - **Prediction resource**: *Your cognitive services resource*
5. If a panel with tips for creating an effective LUIS app is displayed, close it.

### Create an Entity

1. In the LUIS page for your app, in the pane on the left, click **Entities**. Then click **Create new entity**, and create a new entity named **device** of type **List** and click **Next**.
2. In the **Create a list entity** page, click *Add new sublist* and type **light**, then press ENTER.
3. After the **light** entity has been added, under **Synonyms**, click *Type in value*, and type **lamp** and press ENTER.
4. Add a second sublist named **fan** with the synonym **AC**.
5. Click **Create** to create the **device** entity with its **light** and **fan** sublists.

### Create an Intent

1. In the pane on the left, click **Intents**. Then click **Create new intent**, and add an intent with the name **Switch On**.
2. In the **Utterances** page for the **Switch On** intent, type **turn the light on** and press **Enter** to add this utterance to the list.
3. In the list of utterances, in the *turn the light on* utterance, click the word "light", and assign it to the **device** entity.
4. Add a second utterance to the **Switch On** intent, with the phrase **turn the fan on**. Then assign the word "fan" to the **device** entity.
5. In the pane on the left, click **Intents** and click **Create new intent**, to add a second intent with the name **Switch Off**.
6. In the **Utterances** page for the **Switch Off** intent, add the utterance **turn the light off** and assign the word "light" to the **device** entity.
7. Add a second utterance to the **Switch Off** intent, with the phrase **turn the fan off**. Then connect the word "fan" to the **device** entity.

### Train and Test the Language Model

1. At the top of the LUIS page for your app, click **Train** to train the language model
2. After the app has been trained, click **Test**, and then in the test pane, enter the following utterances and inspect the results to verify that they are correctly interpreted as commands for the *Switch On* and *Switch Off* intents with the appropriate entities:
    * *turn on the light*
    * *put the light off*
    * *switch the fan on*
    * *switch the light on*
    * *switch off the light*
    * *turn off the fan*
    
### Publish the Model and Configure Endpoints

1. At the top of the LUIS page for your app, click **Publish**. Then ensure that the **Production** slot is selected and click **Done**.
2. After the model has been published, at the top of the LUIS page for your app, click **Manage**. Then on the **Application Information** tab, note the **Application ID** for your app. Copy this and add paste it in the code below to replace **YOUR_LUIS_APP_ID**.
3. On the **Azure Resources** tab, note the keys and endpoint for your language model, which is based on your cognitive services resource.
  

In [ ]:
import json
import requests
from matplotlib.pyplot import imshow
from PIL import Image
import os
%matplotlib inline

# Set up API configuration
luis_app_id = 'YOUR_LUIS_APP_ID'

# prompt for a command
command = input('Please enter a command: \n')
print(command)

headers = {
}

params ={
    'query': command,
    'subscription-key': cog_key
}

response = requests.get(cog_endpoint + '/luis/prediction/v3.0/apps/'+ luis_app_id + '/slots/production/predict',
                        headers=headers, params=params)
data = response.json()

# Get the most probable intent
intent = data["prediction"]["topIntent"]
print('\tpredicted intent:',intent)

# For simplicity, only the first 'device' entity is identified
entities = data["prediction"]["entities"]
if 'device' in entities:
    device = entities['device'][0][0]
    print('\tpredicted entity:',device)

# Take the appropriate action
if (intent == "Switch On"):
    img = device + "-on.jpg"
elif (intent == "Switch Off"):
    img = device + "-off.jpg"
else:
    img = "Dunno.jpg"
img = Image.open(os.path.join("data", "office-auto" ,img))
imshow(img)